In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from datetime import datetime
import statsmodels.api as sm

# Use Kamppi as an example, could be generalized to all stations
station = 'Kamppi (M)'
exogenous = []

df = pd.read_csv('./datasets/' + station +  "final_data.csv")

df['Departure'] = pd.to_datetime(df['Departure'], format='mixed')
df.set_index(df['Departure'], inplace=True)

df['trip'] = pd.to_numeric(df['trip'], errors='coerce')

days = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat']
for asd in days:
    exogenous.append(asd)

for i in range(0, 23):
    asd = 'hour' + str(i)
    days.append(asd)

df = df.dropna(axis=1)

train_end = datetime(year=2022, month=10, day=31)
train_data = df[:train_end]
test_end = datetime(year=2023, month=10, day = 31)
test_data  = df[datetime(year=2023,month=4,day=1):]

model = sm.tsa.statespace.SARIMAX(train_data['trip'], order=(1,1,0), exog=train_data[exogenous], seasonal_order=(0,0,0,0), freq='h').fit()



/home/aleksi/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.76440D+00    |proj g|=  1.01291D-05


 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8      1      3      1     0     0   5.708D-06   2.764D+00
  F =   2.7643971566110954     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


In [19]:
from datetime import timedelta




dates = list()
date = datetime(year=2022,month=11,day=1, hour=0)
while True:
    if date == datetime(year=2023,month=4,day=1,hour=0):
        break
    dates.append(date)
    date = date + timedelta(hours=1)
    
dateData = pd.DataFrame({'Departure' : dates})
dateData['weekday'] = dateData['Departure'].dt.weekday
days = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat']
for day in days:
    dateData[day] = 0
    dateData.loc[dateData['Departure'].dt.weekday == 0, day] = 1
for i in range(0, 23):
    asd = 'hour' + str(i)
    days.append(asd)
    dateData[asd] = 0
    dateData.loc[dateData['Departure'].dt.hour == i, asd] = 1
dateData['trip'] = 0


test_data.drop(columns=['Departure.1', 'wind_speed_10m (km/h)', 'temperature_2m (°C)', 'rain (mm)'],inplace=True)

test_data = pd.concat([dateData,test_data])
print(test_data.shape)



#forecast = model.predict(start=test_data.index[0], end=test_data.index[-2], exog=test_data[exogenous])
'''

final = pd.concat([forecast,test_data['trip']], axis=1)
errors = final['trip'] - final['predicted_mean']
mse = np.mean(errors**2)
print(mse)
'''

(8758, 33)


/tmp/ipykernel_59732/385537031.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(columns=['Departure.1', 'wind_speed_10m (km/h)', 'temperature_2m (°C)', 'rain (mm)'],inplace=True)


"\n\nfinal = pd.concat([forecast,test_data['trip']], axis=1)\nerrors = final['trip'] - final['predicted_mean']\nmse = np.mean(errors**2)\nprint(mse)\n"

In [17]:
forecast = model.predict(start=datetime(year=2023,month=4,day=1), end=datetime(year=2023,month=10,day=30,hour=22), exog=test_data[exogenous])

final = pd.concat([forecast,test_data['trip']], axis=1)
print(final)
errors = final['trip'] - final['predicted_mean']
mse = np.mean(errors**2)
print(mse)

ValueError: Provided exogenous values are not of the appropriate shape. Required (8758, 6), got (5134, 6).